In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

from datetime import datetime as dt

In [2]:
# Base URL to edit when iterating through dates
BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/'

# States/Provinces to ignore
excluded = ["Guam", "District of Columbia", "Grand Princess",
            "Diamond Princess","Northern Mariana Islands",
            "Puerto Rico", "Recovered", "American Samoa", "Virgin Islands"]

In [3]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [4]:
# Generating Date Ranges for CSV retrieval
dates = pd.date_range(start='04-12-2020', end='07-26-2022').strftime('%m-%d-%Y')
for date in dates[0:5]:
    print(date)
print()
print(f'Total Days: {len(dates)}')

04-12-2020
04-13-2020
04-14-2020
04-15-2020
04-16-2020

Total Days: 836


In [5]:
def generateSmallTables(dates = dates):
    for date in dates:
        url = BASE_URL + date + ".csv"
        print(url)
        df = pd.read_csv(url).fillna(0)
        df.to_csv("data/SmallTables/" + date + ".csv", index=False)

#generateSmallTables()

In [6]:
def editSmallTables(dates = dates):
    for date in dates:
        path = "data/SmallTables/" + date + ".csv"
        print(path)
        df = pd.read_csv(path).fillna(0)
        df['Last_Update'] = date
        df.to_csv("data/SmallTables/" + date + ".csv", index=False)

#editSmallTables()

In [7]:
def generateBigTable(dates = dates):
    # Init main DataFrame
    allData = pd.DataFrame()
    # Iterate through days
    for date in dates:
        path = "data/SmallTables/" + date + ".csv"
        print(path)
        allData = pd.concat([allData, pd.read_csv(path).fillna(0)])
    # There is probably a better way to clean this but it isn't very clear
    allData = allData[allData.Province_State != "Guam"]
    allData = allData[allData.Province_State != "District of Columbia"]
    allData = allData[allData.Province_State != "Grand Princess"]
    allData = allData[allData.Province_State != "Diamond Princess"]
    allData = allData[allData.Province_State != "Northern Mariana Islands"]
    allData = allData[allData.Province_State != "Puerto Rico"]
    allData = allData[allData.Province_State != "Recovered"]
    allData = allData[allData.Province_State != "American Samoa"]
    allData = allData[allData.Province_State != "Virgin Islands"]
    # Drop redundant columns
    allData = allData.drop(["ISO3", "FIPS", "Country_Region", "Date"], axis = 1)
    # Reorder so Primary Key is in front of DataFrame
    uid = allData.pop("UID")
    updated = allData.pop("Last_Update")
    allData.insert(0, "UID", uid)
    allData.insert(1, "Date", updated)
    #allData["Province_State"] = us_state_to_abbrev[allData["Province_State"]]
    allData["Province_State"] = allData["Province_State"].replace(us_state_to_abbrev)

    # Write to CSV
    allData.to_csv("data/bigTable.csv", index=False)

#generateBigTable()

In [8]:
testView = pd.read_csv("data/bigTable.csv")
print(testView.shape)
testView.head(10)

(41800, 17)


,UID,Date,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate
0,84000001.00,04-12-2020,AL,32.32,-86.90,3667,93,0.00,0.00,74.79,0.00,437.00,0.00,1265.08,12.26,62029.00,2.54
1,84000002.00,04-12-2020,AK,61.37,-152.40,272,8,66.00,198.00,37.18,0.00,31.00,0.00,1344.71,11.40,8038.00,2.94
2,84000004.00,04-12-2020,AZ,33.73,-111.43,3542,115,0.00,0.00,48.66,0.00,0.00,0.00,578.52,0.00,52289.00,3.25
3,84000005.00,04-12-2020,AR,34.97,-92.37,1280,27,367.00,886.00,42.41,0.00,130.00,0.00,761.75,10.16,19722.00,2.11
4,84000006.00,04-12-2020,CA,36.12,-119.68,22201,632,0.00,0.00,56.19,0.00,5234.00,0.00,485.42,22.96,190328.00,2.84
5,84000008.00,04-12-2020,CO,39.06,-105.31,7307,289,0.00,0.00,126.89,0.00,1376.00,0.00,615.39,18.83,34873.00,3.96
6,84000009.00,04-12-2020,CT,41.60,-72.76,12035,554,0.00,0.00,337.56,0.00,1654.00,0.00,1156.15,13.74,41220.00,4.60
7,84000010.00,04-12-2020,DE,39.32,-75.51,1625,49,191.00,1385.00,166.88,0.00,190.00,0.00,1140.21,11.69,11103.00,3.02
8,84000012.00,04-12-2020,FL,27.77,-81.69,19895,461,0.00,0.00,92.63,0.00,2772.00,0.00,1508.52,13.93,323996.00,2.32
9,84000013.00,04-12-2020,GA,33.04,-83.64,12452,433,0.00,0.00,117.28,0.00,2505.00,0.00,537.04,20.12,54453.00,3.48


In [9]:
testView.describe()

,UID,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate
count,41800.00,41800.00,41800.00,41800.00,41800.00,41800.00,41800.00,41800.00,41800.00,41800.00,41800.00,41800.00,41800.00,41550.00,41550.00
mean,84000029.32,39.47,-93.67,749196.12,11167.77,30669.81,21378.32,11547.20,8670962.38,754.64,1.10,130538.98,1.47,312579.10,0.84
std,15.62,6.08,19.15,1177240.22,15585.13,123387.51,77075.36,9149.05,17554437.75,5235.15,0.79,127544.81,4.37,1207473.43,1.78
min,84000001.00,21.09,-157.50,270.00,0.00,0.00,0.00,28.74,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,84000017.00,35.63,-105.31,99544.25,1574.00,0.00,0.00,2855.76,0.00,0.00,0.00,26072.18,0.00,0.00,0.00
50%,84000029.50,40.00,-89.65,341046.00,5598.50,0.00,0.00,10274.52,3127878.00,0.00,1.24,105147.64,0.00,0.00,0.00
75%,84000042.00,43.33,-79.81,883742.25,14018.00,4780.50,4544.25,17587.51,9842443.00,0.00,1.58,188195.39,0.00,16097.00,0.69
max,84000056.00,61.37,-69.38,10644446.00,93319.00,2470308.00,1408516.00,58927.99,318233196.00,89995.00,6.28,1635860.89,38.50,19565151.00,9.74


In [33]:
# Age and Sex Data
path = "data/SexAndAge/COVID-19_Death_Counts_by_Age_2020-2022.csv"
sex_age = pd.read_csv(path)
sex_age.head()

sex_age = sex_age.drop(["Data as of", "Start Date", "End Date"], axis=1)
sex_age.to_csv(path, index=False)
sex_age.head(200)

,Sex,Age Years,Total deaths,COVID-19 Deaths
0,Female,0-05 Months,19659,103
1,Male,0-05 Months,24188,126
2,Female,06-11 Months,1639,32
3,Male,06-11 Months,2111,45
4,Female,01 Year,1567,34
...,...,...,...,...
169,Male,83 Years,107411,14379
170,Female,84 Years,111326,12230
171,Male,84 Years,105517,13767
172,Female,85 Years and over,1483023,148112


In [27]:
# Load Vaccine Data
path = "data/vaccine2.csv"
vaccine = pd.read_csv(path, low_memory=False).fillna(0)
vaccine = vaccine[vaccine.ID == "US"]
#vaccine.drop(["Unnamed: 0.1"], axis=1)
vaccine.to_csv(path, index=False)
print(vaccine.shape)
vaccine.head()

(862225, 8)


,ID,FIPS,Date,Vaccine,DoseType,DoseValue,Vax_Full,Vax_Partial
186350,US,0,12/20/2020,0,0,0.00,0.00,556208.00
186351,US,0,12/21/2020,0,0,0.00,0.00,614117.00
186352,US,0,12/22/2020,0,0,0.00,0.00,614117.00
186353,US,0,12/23/2020,0,0,0.00,0.00,1008025.00
186354,US,0,12/24/2020,0,0,0.00,0.00,1008025.00


In [28]:
vaccine.describe()

,DoseValue,Vax_Full,Vax_Partial
count,862225.00,862225.00,862225.00
mean,1835769.82,3351539.61,903544.30
std,11176219.64,7554222.48,7520687.33
min,-271346.00,0.00,0.00
25%,0.00,621891.00,129232.00
50%,4268.00,1730579.00,351564.00
75%,1126723.00,3898524.00,655038.00
max,587903405.00,223245563.00,261654261.00
